$$
\newcommand{proof}{\textbf{Proof: }}
\newcommand{theorem}{\textbf{Theorem: }}
$$

# Reduction

**Reducing** problem $X$ to problem $Y$ means to write an algorithm that solves $X$ using an algorithm for $Y$.


## Merge sort
Merge sort is defined as below

We know that this is correct because we know that it is correct in the base case.
Secondly, if the two subarray is sorted, then the `merge` function would produce a sorted array.
Using these 2 information, we know that the algorithm is correct by induction.

## Quick sort
Quick sort algorithm is as below

In [28]:
from random import randint

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    
    def partition(arr, index):
        pivot = arr[index]
        return [e for i, e in enumerate(arr) if e <= pivot and i != index], [e for e in arr if e > pivot]
    index = randint(0, len(arr) - 1) 
    left, right = partition(arr, index)
    left = quick_sort(left)
    right = quick_sort(right)
    return left + [arr[index]] + right
            

In [29]:
quick_sort([1, 2, 4, 5, 3, 9, 4, 2])

[1, 2, 2, 3, 4, 4, 5, 9]

Similar to merge sort, we split the array into 2 parts.
Then we use our sorting routine on the smaller array.
Since we know that the resultant array will be sorted if the two subarray is sorted, and the base case is defined, we know that quick sort is also correct.

### Analysis
The recurrence we get depends on $r$, the rank of the pivot.
We get
$$
T(n) = T(r-1) + T(n-r) + O(n)
$$

If $r$ is always $1$ or $n$ for all the routines, then we can compute the complexity and get $O(n^2)$, which means quick sort is rather inefficient when the pivot is chosen to be the ends of the array.


And if we somehow can choose a pivot with rank $n/2$ (which is the median), then the complexity reduces to 

$$
T(n) = T(\lceil n/2 \rceil - 1) + T(\lfloor n/2 \rfloor) + O(n) \leq 2T(n/2) + O(n)
$$


And we can compute that the complexity is $O(n \log n)$.

## Quick select
Suppose instead, that we wish to find the element with a given rank $r$ in the array.


## Good pivot
1. Partition the array into groups of 5 (pad with infinity if needed)
2. Sort each group and find the median
3. The pivot is the median of these medians

Notice that there will be $3n/10$ elements that are smaller than the chosen pivot.
Plugging $r=3n/10$ into our recursive formula for our quick sort, we get

$$
T(n) \leq T(7n/10) + T(n/5) + Cn
$$
where $T(7n/10)$ is the complexity of the subproblem, and $T(n/5)$ is the complexity of sorting the groups of 5.
we notice that the complexity is $\log n$.

## Integer multiplication
We have long taken for granted that the time complexity to multiply any 2 numbers is $O(1)$.
What if this assumption does not hold for large numbers?
How would we derive a multiplication algorithm for larger numbers that our builtin integer multiplication can no longer support?

Suppose that we have 2 $n$-digit integers that we need to multiply, and adding or multiply two **digits** is $O(1)$.
How would we compute their product?

Suppose the number that we want to multiply is represented as $a_1, a_2, \dots, a_n$ and $b_1, b_2, \dots , b_n$ in base 10.
The naive way would be to multiply $a_n$ with $b_1, b_2, \dots b_n$,
then multiply $a_{n-1}$ by $b_1, b_2, \dots b_n$, then appending a $0$ at the end, 
then multiply $a_{n-2}$ by $b_1, b_2, \dots b_n$, then appending a $00$ at the end, and so on.
Then, we sum all the products.
This would require $n$ multiplications of a 1-digit number with a $n$-digit number, thus $O(n^2)$.
And since there will be $n$ products, the summation would take $O(n)$.
Thus, the overall complexity would be $O(n^2)$.

First, notice that we have the following identity
$$
pq = (10^m a + b)(10^m c + d) = 10^{2m} ac + 10^m{ad + bc} + bd
$$

Notice that to solve $pq$, we need to simply find $ac, ad, bc$ and $bd$.
And it inspires us that we can compute these recursively.

Computing the complexity, we get 
$$
T(n) = 4T(\lceil n/2 \rceil) + O(n)
$$

Using master theorem, we get $O(n^2)$.

Note that we did not actually need $ad$ and $bc$, but what we actually need is $ac + bd$.
Notice that 
$$
ad + bc = ac + bd - (a-b)(c-d)
$$

And the solution become apparent, we recursively compute $(a-b)(c-d)$ instead of $ad$ and $bc$.
This means the number of multiplication we need is now 3.


Recomputing the complexity, we get 
$$
T(n) = 3T(\lceil n/2 \rceil) + O(n)
$$

Using master theorem, we get $O(n^{\log_2 3})$.

## Worst-case time and average time
For some given input size $n$, the **worst case time** is defined as the maximum time needed among all possible input.
And the **average** time is the time needed over some finite set of input.
Usually, in algorithm analysis, we are interested in the worst-case complexity rather than the average complexity.

In the above algorithm, we have 2 nested loops.
The inner loop has 3 assignments (including the loop itself), and is executed $n$ times, once for each element in the array.
Thus, the complexity of the inner loop would be $3n + 1$, as we need to execute the `for` statement one last time before we exit the loop.

Now, the outer loop executes the inner loop $n$ times too, which means the outer loop has the complexity of $(3n+1)\times n + n + 1= 3n^2 + 2n + 1$.
Lastly, the rest of `foo` has 3 more assignments, if you include the `return`, thus the total complexity would be $f(n) = 3n^2 + 2n + 4$.

Indeed, when we add our count tracking into our algorithm we get $3 \times 5^2 + 2 \times 5 + 4 = 89$ executions on an array of length $5$.

## Strassen
TODO